# Projektphase 1


## Lastenheft Aufgabe 10


### Dash Stufe 3

In [3]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
import pandas as pd
import numpy as np
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import datetime
import plotly.express as px

df = pd.read_csv('Log_SensorCar_USS_n.csv')

speed_max = df['Geschwindigkeit'].max()
speed_min = df['Geschwindigkeit'].min()
speed_mean = df['Geschwindigkeit'].mean()
speed_mean = np.round(speed_mean, decimals = 2)

distance = 0
time_sum = 0

drive_time = pd.Series([], dtype = 'float64')
drive_distance = pd.Series([], dtype = 'float64')

for i in range(len(df)):
    speed = df['Geschwindigkeit'][i]
    
    if i <= 2:
        time = 0.0
    else:
        time_1 = df['Zeit'][i-1]
        time_1 = datetime.datetime.strptime(time_1, '%H:%M:%S.%f')

        time_2 = df['Zeit'][i]
        time_2 = datetime.datetime.strptime(time_2, '%H:%M:%S.%f')

        time = time_2 - time_1
        time = time.total_seconds()
        
        if time >= 1 and speed == 0:
            time = 0.0
           
    distance = np.round((distance + time*speed), decimals = 2)
    time_sum = np.round((time_sum + time), decimals = 2)
    
    drive_time[i] = time_sum
    drive_distance[i] = distance

df.insert(5,'Gefahrene_Zeit', drive_time, True)
df.insert(6,'Gefahrene_Distanz', drive_distance, True)

app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])


speed_max_card = html.Div(
    [
        dbc.Card(
            dbc.CardBody("Maximale Geschwindigkeit"),
            className="card-text",
        ),
        dbc.Card(speed_max, body=True),
    ]
)

speed_min_card = html.Div(
    [
        dbc.Card(
            dbc.CardBody("Minimale Geschwindigkeit"),
            className="card-text",
        ),
        dbc.Card(speed_min, body=True),
    ]
)

speed_mean_card = html.Div(
    [
        dbc.Card(
            dbc.CardBody("Durchschnittsgeschwindigkeit"),
            className="card-text",
        ),
        dbc.Card(speed_mean, body=True),
        
    ]
)

distance_card = html.Div(
    [
        dbc.Card(
            dbc.CardBody("Gefahrene Distanz"),
            className="card-text",
        ),
        dbc.Card(distance, body=True),
    ]
)

time_sum_card = html.Div(
    [
        dbc.Card(
            dbc.CardBody("Gefahrene Zeit"),
            className="card-text",
        ),
        dbc.Card(time_sum, body=True),
    ]
)

app.layout = html.Div(
    children=[
        html.H1(id='H1',
                children='Auswertung des Log-Files',
                style={'textAlign': 'center', 'marginTop': 40, 'marginBottom': 40}),
        html.H2(id='H2',
                children='Log_SensorCar_USS.csv'),
        html.Br(),
        dbc.Row([
            dbc.Col([speed_max_card], width=2),
            dbc.Col([speed_min_card], width=2),
            dbc.Col([speed_mean_card], width=2),
            dbc.Col([distance_card], width=2),
            dbc.Col([time_sum_card], width=2),
        ], align='center'),
        html.Br(),
        html.Div(children='Auswahl Fahrdaten:'),
        html.Br(),
        dcc.Dropdown(id='dropdown',
                     options=[
                         {'label': 'Geschwindigkeit', 'value': 'Geschwindigkeit'},
                         {'label': 'Fahrtrichtung', 'value': 'Fahrtrichtung'},
                         {'label': 'Lenkwinkel', 'value': 'Lenkwinkel'},
                         {'label': 'Abstand_Hindernis', 'value': 'Abstand_Hindernis'},
                         {'label': 'Gefahrene_Zeit', 'value': 'Gefahrene_Zeit'},
                         {'label': 'Gefahrene_Distanz', 'value': 'Gefahrene_Distanz'},
                     ],
                     value='Geschwindigkeit'),
        html.Br(),
        dcc.Graph(id='line_plot'),
    ]
)

# Callback für den Plot als Ausgabe (siehe 'line_plot') und den Wert des Dropdown Menüs als Eingabe
@app.callback(Output(component_id='line_plot', component_property='figure'),
              [Input(component_id='dropdown', component_property='value')])
def graph_update(value_of_input_component):
    print(value_of_input_component)
    fig = px.line(df, x=df['Zeit'], y=df[value_of_input_component])
    return fig

if __name__ == '__main__':
    app.run_server()


OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.